<a href="https://colab.research.google.com/github/br4bit/Neural-Network-Training/blob/master/Training_Deep_Neural_Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Addestramento rete neurale profonda (Deep Neural Network)

Questo tipo di addestramento richiede un'analisi accurata dei seguenti problemi:



*  Primo: Problema del *Vanishing Gradients* e il correlato *Exploding Gradients* che si presenta nelle reti neurali molto profonde (più di 10 layers), questo problema rende difficile l'allenare i layers che si trovano nello strato inferiore.
*   Secondo: L'addestramento della rete neurale rallenta di molto con reti molto grandi.

*  Terzo: Un modello con milioni di parametri rischirebbe gravemente l'overfitting sul set di training.



# Problemi del Vanishing/Exploding Gradients

L'algoritmo di backpropagation lavora andando dall'output layer all'input layer, propagando l'errore del gradiente durante il tragitto. Una volta che l'algoritmo ha calcolato il gradiente della cost function utilizzando ogni parametro della rete, i parametri della rete vengono aggiornati (Gradient Descent step).

Sfortunatamente i gradienti calcolati diventano via via, sempre più piccoli man mano che l'algoritmo procede nei layers inferiori. Come risultato l'aggiornamento del Gradiente Descent, lascia i pesi delle connessioni nel layer inferiore virtalmente immutati (non li cambia), e l'addestramento non converge quindi ad una buona soluzione, da qui il nome *Vanishing Gradients*. In molti casi però, può accadere il contrario, ovvero i gradienti (derivate parziali) possono diventare molto grandi, e quindi causare anche l'incremento dei pesi in alcuni layer, di conseguenza l'algoritmo diverge (*Exploding Gradients*).
Più in generale le reti neurali, soffrono di gradienti instabili, dove diversi layers possono imparare a velocità ampliamente differenti. Questo è uno dei motivi per cui sono state abbandonate per molto tempo.

Ma nel 2010 grazie a : [Understanding the difficulty of training deep feedforward neural networks](http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf)

Che attraverso dei test con le funzioni di attivazione e inizializzazione dei pesi utilizzando una distribuzione normale con media 0 e deviazione standard a 1. In breve è stato dimostrato che con quel tipo di funzione di attivazione e schema di inizializzazione, la varianza dell'output di ogni layer era molto più grande della varianza dei proprio inputs. Andando in avanti nella rete la varianza continuava a crescere ad ogni layer fino a che la funzione di attivazione assumeva valori di saturazione nei layer in cima. Questo in realtà è aggravato dal fatto che la funzione logisitica ha una media di 0.5, non 0! (ad esempio la funzione tangente hyperbolica ha media 0 e si comporta molto bene rispetto alla logistic nelle reti molto grandi).

Diamo un'occhiata alla funzione logistica di attivazione (Sigmoid):

![alt text](https://www.simplilearn.com/ice9/free_resources_article_thumb/gradients-in-sigmoid-activation-functions.jpg)

E' possibile vedere che quando l'input assume valori molto grandi (sia positivi che negativi), la funzione satura o meglio assume valore 1 o 0, e in quei punti la pendenza della retta tangente (la derivata) assume valori estremamente vicino allo 0.
Ne consegue che durante la fase di backpropagation, non esiste virtualmente nessun gradiente da propagare attraverso la rete, e quel piccolo gradiente che esiste si diluisce, mentre il backpropagation progredisce verso il basso attraverso i layers superiori, quindi non rimane nulla per i livelli inferiori.

## Xavier and He Inizializzazione

Dall'articolo Glorot e Bengio hanno proposto un modo per alleviare significamente questo problema. C'è bisogno che il segnale fluisca in modo appropriato in entrambe le direzioni: In avanti quando si fa una predizione e indietro quando si backpropagano i gradienti. Il segnale quindi non deve morire o assumere valori di saturazione.

Per far si che il segnale fluisca in modo corretto, l'autore ci dice che la varianza degli output di ogni layer deve essere uguale alla varianza degli input del layer : Var_Out(layer[1] = Var_In(layer[1])), questo per ogni layer. Inoltre ce bisogno che i gradienti abbiano prima e dopo l'attraversamento della rete nella direzione opposta, la stessa varianza. Non è possibile però garantire entrambi le condizioni a meno che i layers non abbiano lo stesso numero di connessioni di input e output, ma è possibile arrivare ad un compromesso che funziona molto bene in pratica:

Le connessioni dei pesi (matrice W) devono essere inizializzati in modo casuale secondo l'equazione di Xavier o Glorot.

![alt text](https://www.simplilearn.com/ice9/free_resources_article_thumb/normal-distribution-with-0-mean-and%20standard-deviation-formula.jpg)

Usando una delle due inizializzazione è possibile velocizzare l'addestramento. Inoltre questa strategia di inizializzazione nel caso si decida di utilizzare la funzione ReLU come funzione di attivazione è chiamata *He Initialization*.

Di default la funzione *tf.layers.dense()* usa la Xavier con distribuzione uniforme, è possibile cambiare anche con l'inizializzazione di He usando *variance_scaling_initializer()*

In [0]:
he_init = tf.contrib.layers.variance_scaling_initializer()

hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu(), 
                          kernel_initializer=he_init, name="hidden1")


## Non Saturazione della funzione di attivazione

Sfortunatamente la funzione di attivazione ReLU non è perfetta, ma soffre di un problema conosciuto come *dying ReLus*, durante l'addestramento (training), alcuni neuroni è come se morissero, o meglio smettono di dare come output valori che sono diversi da 0, specialmente se si usa un learning rate molto alto. 

Per risolvere questo problema si usano delle varianti della funzione ReLU, come la *leaky ReLU*, questa funzione è definita come:

![alt text](http://img.thothchildren.com/09d44cab-d699-402a-af31-fdced34d94fd.png)
![alt text](http://wangxinliu.com/images/machine_learning/leakyrelu.png)

Dove α è un Hyperparametro e definisce quanto deve perdere la funzione o meglio qual è la pendenza della funzione per x<0 e tipicamente è impostato a 0.01. Questa piccola pendenza ci assicura che la Leaky ReLU non muoia mai.

Esistono molto varianti di questo genere, in particolare una nuova funzione di attivazione chiamata ELU (Exponential linear Unit) che ha superato la performance di tutte le varianti di ReLU nei vari esperimenti: tempo di training ridotto e performance migliorata sul test test. 

Ecco la sua definizione e rappresentazione:

![alt text](https://i.gyazo.com/7851e87ec39ce6a960e26709f39c1825.png)

![alt text](https://blog.paperspace.com/content/images/2018/06/ELU.png)

E' molto simile alla ReLU ma con alcune differenze:

* Primo: Ha una regione di valori negativi quando x<0, che permette alle unità di assumere valori di output vicino allo 0, il che aiuta con il problema del vanishing gradient. Hyperparamentro  α definisce il valore che la ELU deve approcciare quando x è un numero negativo molto grande, usualmente è impostato a 1.
* Secondo: Il suo gradiente per x<0 non è zero, quindi si evita il problema dei neuroni morti.
* Terzo: Quando Hyperparametro è uguale a 1 la funzione è molto regolare/morbida ovunque, incluso nell'intorno di x=0, questo aiuta molto a velocizzare il calcolo dei gradienti, evitando saltelli a destra e a sinistra nell'intorno di x=0.

L'unica pecca è che la ELU impiega molto tempo computazionale, dovuto alla funzione esponenziale, ma questo tempo è compensato dal fatto che durante il training converge molto più velocemente, quindi una rete con ReLU risulta di poco più veloce.

Quindi, quale funzione di attivazione utilizzare per gli hidden layers?

In generale ELU > leaky RELU (e varianti) > ReLU > tanh > logistic.

Se ci interessa il tempo di performance durante il runtime, la leaky ReLU è preferibile rispetto alla ELU.

Ma se il tempo di performance non ci interessa molto e abbiamo potenza computazione elevata è possibile utilizzare altre funzioni nel caso in cui: RReLU (overfitting) oppure PReLU se abbiamo un training set molto grande.

TensorFlow ci offre la *elu()* che è possibile utilizzare per costruire il nostro modello di rete neurale. Semplicemente basta impostare l'argomento del campo *activation*.

In [0]:
hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.elu, name="hidden1")

Sfortunatamente TensorFlow non ha una funzione predefinita per la leaky ReLU, ma è possibile definirne una, nel seguente modo:

In [0]:
def leaky_relu(z, name=None):
  return tf.maximum(0.01 * z, z, name=name)

hidden1 = tf.layers.dense(X, n_hidden1, activation=leaky_relu, name="hidden1")

## Batch Normalization

Anche se usando l'inizializzazione dei pesi con il metodo di He, si ha una riduzione del problema del vanishing/exploding gradients all'inizio dell'addestramento, non è garantito che in seguito questo problema non si presenti, durante il training della rete.

Una tecnica scoperta nel 2015 chiamata *Batch Normalization (BN)* affronta il problema in modo diretto o più in generale, il problema della distribuzione di ogni input dei layer, che cambia durante il training a seconda di come cambiano i parametri dei layer precedenti, questo cambiamento della distribuzione è noto come *Internal Covariate Shift*.

La tecnica consiste nell'aggiungere un'operazione nel modello prima che venga eseguita la funzione di attivazione di ogni layer, semplicemente centrando a zero gli input e normalizzandoli, scalando e shiftando il risultato, usando due nuovi parametri per layer (uno per lo scaling e l'altro per lo shifting). In poche parole questa operazione permette al modello di imparare lo scale e la media ottimale degli inputs per ogni layer, in modo da rendere la distribuzione degli input più uniforme, riducendone la varianza o variazione.

L'algoritmo ha bisogno di conoscere per ogni mini-batch di input, la media e la deviazione standard.
Tutte le operazioni che esegue l'algoritmo:


>![alt text](https://i.gyazo.com/45070cbcbd577a47f40cb04034784fc9.png)



1. Calcolo della media  e della varianza degli input dei layer:
La media è calcolata su tutti i campioni del mini-batch B.

2. Varianza calcolata su tutto il mini-batch, dove mB è il numero dei campioni o istanze presenti nel mini-batch.

3. E' l'input normalizzato e centrato in zero.

4. Gamma è il parametro di scaling per il layer.

5. Beta è il parametro di shifting (offset) per il layer.

6. Epsilon è un numero arbitrario piccolo per evitare la divisione per zero (tipicamente 10-5) è chiamato *Smoothing Term*.

7. **z(i)** è l'output dell'operazione di Batch Normalization (BN): non è nient'altro che una versione shiftata e scalata degli input.

Durante la valutazione con il test set, non esiste nessun mini-batch, quindi si usa la media e la deviazione standard calcolate su tutto il training set. In totale si hanno quattro parametri per ogni batch normalizzato: gamma (scale), beta(offset), mu(media) e sigma (deviazione standard).

Con questa tecnica si riduce considerevolmente il problema del vanishing/exploding gradients al punto di permettere l'uso delle funzione di attivazioni che hanno problemi di saturazione come tanh,logist,ReLU,ecc.
La rete risulta inoltre meno sensibile all'inizializzazione dei pesi, ed è possibile usare anche un learning rate alto, velocizzando quindi il processo di addestramento (meno step e precisione più elevata).

Ma come tutte le tecniche viste fino ad ora, anche il Batch Normalization introduce una ulteriore complessità al modello, la penalizzazione in runtime: La rete neurale effetua delle predizioni lenti a causa della computazione extra richiesta per ogni layer, deve effettuare il BN (il primo layer quello di input non viene normalizzato, i dati vengono normalizzati a partire dal primo hidden layer), quindi se si ha bisogno di una rete che effettua delle predizioni veloci sui dati, prima di implementare il Batch Normalization è opportuno dare un'occhiata delle performance pianificando al meglio la ELU + He Initialization.

### Implementazione del Batch Normalization in TensorFlow

TensorFlow ci viene in aiuto anche in questo caso, mettendo a disposizione la funzione *tf.nn.batch_normalization()* che semplicemente centra e normalizza gli inputs, ma tocca a noi calcolare la media e la deviazione standard (basate sul mini-batch di dati corrente durante la fase di training oppure su tutto il dataset durante la fase di testing) e passarle come parametri a questa funzione, e bisogna anche gestire la creazione dei parametri di scaling e shifting(offset), passando anche questi alla funzione. **Non è un approccio molto conveniente**.

La funzione *tf.layers.batch_normalization()* si occupa di eseguire tutte queste operazioni.
Riprendiamo il modello della guida precedente, abbiamo:

In [0]:
import tensorflow as tf

n_inputs = 28*28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None,n_inputs), name="X")

training = tf.placeholder_with_default(False,shape=(), name='training')

#Costruzione del primo hidden layer e layer di BN
hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1")
bn1 = tf.layers.batch_normalization(hidden1, training=training, momentum=0.9)
bn1_act = tf.nn.elu(bn1) #output del layer normalizzato

#Costruzione del secondo hidden layer e layer di BN
hidden2 = tf.layers.dense(bn1_act, n_hidden2, name="hidden2")
bn2 = tf.layers.batch_normalization(hidden2, training=training, momentum=0.9)
bn2_act = tf.nn.elu(bn2)

#Costruzione del layer di output
logits_before_bn = tf.layers.dense(bn2_act, n_outputs, name="outputs")
logits = tf.layers.batch_normalization(logits_before_bn, training=training, momentum=0.9)




**Opzionale:** La prima parte del codice è quella vista precedetemente, fino ad arrivare alla definizione della variabile *training*, questa variabile sarà settata a True durante la fase di training, e serve per dire alla funzione *tf.layers.batch_normalization()* quando usare la deviazione standard e media calcolate sul mini-batch corrente durante la fase di training, e quando usare la media e deviazione standard dell'intero training set, durante la fase di testing.

Successivamente si costruiscono gli hidden layers e l'output layer.
Per gli hidden layes si nota che durante la chiamata della funzione *tf.layers.dense()* non è utilizzato il parametro *activation* questo perchè vogliamo applicare la funzione di attivazione soltanto dopo il batch normalization del layer. Creiamo i batch normalization layers con la funzione *tf.layers.batch_normalization()* impostando i parametri training e momentum. L'algoritmo di BN usa la tecnica del decadimento dell'esponenziale per calcolare le medie correnti, ecco perchè è richiesto anche il parametro *momentum*, dato un nuovo valore v, le medie correnti v' vengono aggiornate attraverso l'equazione:

v' <- v' * momentum + v x (1- momentum).

Un buono valore per il momentum è tipicamente vicino a 1 (0.9 , 0.99, 0.999), per dataset molto grandi e mini-batch piccoli è preferibile usare più 9 dopo lo 0.



---



Il codice risulta ripetitivo, con i parametri di BN che appaiono ovunque, per evitare questa ripetizione di linee di codice, usiamo la funzione *partial* dal modulo *functools* che crea un involucro attorno alla funzione e ci permette di usarla in modo semplice. Modificando così la creazione dei layers in questo modo:

In [0]:
from functools import partial

my_batch_norm_layer = partial (tf.layers.batch_normalization, training=training, momentum=0.9)

hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1")
bn1 = my_batch_norm_layer(hidden1)
bn1_act = tf.nn.elu(bn1) #funzione di attivazione sul batch normalization

hidden2 = tf.layers.dense(bn1_act, n_hidden2, name="hidden2")
bn2 = my_batch_norm_layer(hidden2)
bn2_act = tf.nn.elu(bn2)

logits_before_bn = tf.layers.dense(bn2_act, n_outputs, name="outputs")
logits = my_batch_norm_layer(logits_before_bn)

Se si hanno 10 layer e si vuole usare la stessa funzione di attivazione, inizializzazione, regolarizzazione su tutti i layer, questo piccolo trucchetto renderà il codice più leggibile.

Il resto di costruzione del modello è quello visto nella guida precedente: definizione della cost function e loss, creazione dell'optimizer e GradientDescent per minimizzare la cost function, definizione della valutazione del modello, creazione della variabile initializer (init), creazione del *Saver*.

La fase di esecuzione è molto simile, con due eccezioni.
Primo: ogni volta che durante la fase di training si esegue un'operazione che dipende dal layer *batch_normalization()*, bisogna impostare la variabile di training a True.
Secondo: la funzione di *batch_normalization()* crea delle operazioni che bisogna valutare ad ogni step durante il training in modo da aggiornare la media dei mini-batch. 
Queste operazioni sono automaticamente aggiunte  alla collezione *UPDATE_OPS* , tutto quello che bisogna fare e prendere la lista delle operazioni in quella collezione ed eseguirle ad ogni iterazione durante il training

In [0]:
extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

with tf.Session() as sess:
  
  init.run()
  
  for epoch in range(n_epochs):
    
    for iteration in range(mnist.num_examples // batch_size):
    
      X_batch, y_batch = mnist.train.next_batch(batch_size)
      sess.run([training_op,extra_update_ops], feed_dict={training=True, X: X_batch, y: y_batch})
    
    accuracy_val = accuracy.eval(feed_dict={X: mnist.test.images, y: mnist.test.labels})
    print(epoch, "Test Accuracy: ",accuracy_val)
    save_path = saver.save(sess,"./my_model_final.ckpt")

## Gradient Clipping

Questa tecnica veniva utilizzata prima del BN e consiste nell'effettuare una operazione di "clip" del gradiente durante la backpropagation in modo che non superi mai i valori di soglia, è molto usato nelle RNN.
In TensorFlow la funzione *minimize()* si occupa di eseguire il clipping al posto nostro, dall'immagine è possibile vedere cosa accade durante il calcolo del gradiente:

![alt text](https://i.gyazo.com/a15e2c281064f74a3ea44c48a0f0b96c.png)

# Riuso dei layers già addestrati

In generale non è una buona idea addestrare una rete neurale molto grande da zero, richiede una potenza computazionale molto elevata.
Si può, invece, trovare una rete neurale che adempisca allo stesso tipo di problema che vogliamo modellare, semplicemente usando i layer inferiori della rete, in genere questo tipo di approccio al problema è chiamato *transfer learning*, che non solo aiuta a velocizzare l'addestramento della rete ma richiede anche un numero di dati inferiori per l'addestramento.

Ad esempio, supponiamo di avere un modello di rete neurale che è stata addestrata e utilizzata per classificare le immagini in 100 differenti categorie, animali,piante,veicoli,ecc. E' possibile utilizzare questa rete per classificare specifici tipi di veicoli. Questi compiti sono molto simili, quindi è possibile utilizzare le parti della rete:

![alt text](https://i.gyazo.com/4eb57033333fb178c67a5479012e55bd.jpg)

*Nota: Se l'immagine di input del nostro nuovo task, non ha la stessa grandezza di quelle utilizzate per il task originale, bisogna aggiungere una fase di preprocesso dei dati, dove si ridimensionano i dati in modo da farli coincidere con quelli utilizzati nel modello originale, in generale il transfer learning funziona bene se gli input hanno lo stesso tipo di componenti nei livelli inferiori.*

## Riuso modello in TensorFlow

Se il modello originale è stato addestrato usando TensorFlow, è possibile ripristinare e addestrare il modello per un nuovo task. 
Per prima cosa bisogna caricare le operazioni con la function *import_meta_graph()* , questa funzione ritorna un *Saver* che è utilizzato per caricare lo stato del modello:

In [0]:
saver = tf.train.import_meta_graph("./my_model_final.ckpt.meta")

Ora bisogna ottenere una gestione delle operazioni e dei tensori necessari per l'addestramento, utilizzando i metodi: *get_operation_by_name()* e *get_tensor_by_name()*.
Il nome del tensore è il nome dell'operazione dove l'output è seguito da :0 (o :1 se è il secondo output, :2 se è il terzo output, e così via):

In [0]:
X = tf.get_default_graph().get_tensor_by_name("X:0")
y = tf.get_default_graph().get_tensor_by_name("y:0")

accuracy = tf.get_default_graph().get_tensor_by_name("eval/accuracy:0")

training_op = tf.get_default_graph().get_operation_by_name("GradientDescent")

Se il modello pre-addestrato non è ben documentato, bisogna esplorare i grafi e trovare i nomi delle operazioni di cui abbiamo bisogno. In questo caso è possibile esplorare il grafo usando TensorBoard oppure il metodo *get_operations()* per avere una lista di tutte le operazioni: 

In [0]:
for op in tf.get_default_graph().get_operations():
  print(op.name)

Un altro approccio è quello di creare una collezione che contiene tutte le operazioni importanti in modo da rendere più facile il caricamento del modello successivamente:

In [0]:
for op in (X, y, accuracy, training_op):
  tf.add_to_collection("my_important_ops",op)

In questo modo, il riuso del modello è semplice, basta caricare tutte le operazioni senza andare ad esplorare il grafo:

In [0]:
X, y, accuracy, training_op = tf.get_collection("my_important_ops")

Infine, lo stato del modello utilizzando *Saver* e continuando l'addestramento sui propri dati:

In [0]:
with tf.Session() as sess:
  saver.restore(sess,"./my_model_final.ckpt")
  [...] #Addestramento del modello sui propri dati

Esempio completo:

In [0]:
reset_graph()

n_hidden4 = 20  # nuovo layer
n_outputs = 10  # nuovo layer

saver = tf.train.import_meta_graph("./my_model_final.ckpt.meta")

X = tf.get_default_graph().get_tensor_by_name("X:0")
y = tf.get_default_graph().get_tensor_by_name("y:0")

hidden3 = tf.get_default_graph().get_tensor_by_name("dnn/hidden3/Relu:0")

new_hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.relu, name="new_hidden4")
new_logits = tf.layers.dense(new_hidden4, n_outputs, name="new_outputs")

with tf.name_scope("new_loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=new_logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("new_eval"):
    correct = tf.nn.in_top_k(new_logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

with tf.name_scope("new_train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()
new_saver = tf.train.Saver()

Analizziamo il codice:
Con *import_meta_graph()* si carica l'intero grafo ed è possibile ignorare semplicemente la parte che non interessa. In questo esempio si aggiungono 2 nuovi layer, il 4 hidden layer e l'output layer che si trovano al livello superiore della rete preaddestrata.
Infinire si definisce una nuova loss function, un nuovo metodo di valutazione e di ottimizzazione.
Cè bisogno di un ulteriore saver per salvare il nuovo grafo che contiene entrambi, il vecchio e le nuove operazioni e init per inizializzare le nuove variabili.

In [0]:
with tf.Session() as sess:
    init.run()
    saver.restore(sess, "./my_model_final.ckpt")

    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Validation accuracy:", accuracy_val)

    save_path = new_saver.save(sess, "./my_new_model_final.ckpt")

In ultimo si addestra il modello, ripristinando la sessione.

Se si ha accesso al codice python che è stato usato per costruire il modello originale, è possibile riusare le parti che interessano e scartare il resto:

In [0]:
reset_graph()

n_inputs = 28 * 28  # MNIST
n_hidden1 = 300 # reused
n_hidden2 = 50  # reused
n_hidden3 = 50  # reused
n_hidden4 = 20  # new!
n_outputs = 10  # new!

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1")       # reused
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name="hidden2") # reused
    hidden3 = tf.layers.dense(hidden2, n_hidden3, activation=tf.nn.relu, name="hidden3") # reused
    hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.relu, name="hidden4") # new!
    logits = tf.layers.dense(hidden4, n_outputs, name="outputs")                         # new!

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In questo caso i layer 1,2,3 sono stati riutilizzati e sono stati aggiunti 2 nuovi layer, il 4 e l'output layer al pezzo di codice del modello originale.
Tuttavia, bisogna creare un *Saver* per ripristinare il modello preaddestrato in modo da caricare le variabili necessarie. Un altro *Saver* per salvare i progressi del nuovo modello.

In [0]:
reuse_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,
                               scope="hidden[123]") # regular expression
restore_saver = tf.train.Saver(reuse_vars) # to restore layers 1-3

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    init.run()
    restore_saver.restore(sess, "./my_model_final.ckpt")

    for epoch in range(n_epochs):                                            
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size): 
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})        
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})     
        print(epoch, "Validation accuracy:", accuracy_val)                   

    save_path = saver.save(sess, "./my_new_model_final.ckpt")

Analizziamo il codice:

Nella prima parte è stato costruito il nuovo modello sulla base di quello originale.
Nella seconda parte usando l'espressione regolare "hidden[123]" si ha una lista di tutte le variabili degli hidden layer 1,2,3. La variabile reuse_vars non è nient'altro che un dizionario che mappa il nome di ogni variabile del modello originale nel nome del nuovo modello.
Infine si crea un *Saver* che ripristina queste variabili. Si crea anche un operation per inizializzare tutte le variabili (vecchie e nuove) e un secondo *Saver* per salvare il nuovo modello, non solamente i layer 1,2,3.
Infine si starta la session dove si inizializzazione tutte le variabili del modello e si ripristina il valore delle variabili dei layer 1,2,3 dal modello originale. Il modello finale è addestrato sul nuovo task e salvato.

*Nota: Più i task sono simili più layers è possibile riusare (iniziando dai layers inferiori). Per ogni task simile si cerca di tenere gli hidden layer del vecchio modello, rimpiazzando solo l'output layer.*

## Freezing dei layers inferiori

E' un'ottima idea effettuare il "Freeze" dei pesi dei layer inferiori quando si addestra un nuovo modello di rete neurale a partire da uno già addestrato, in modo da rendere facile l'addestramento dei layer al livello superiore aggiunti da noi. Per congelare i layers di livello inferiore durante il training, bisogna dare all'optimizer la lista delle variabili da usare escludendo le variabili dei layer inferiori:

In [0]:
train_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="hidden[34]|outputs")

training_op = optimizer.minimize(loss, var_list=train_vars)

La prima riga prende una lista di tutte le variabili da addestrare dagli hidden layer 3 e 4 e dall'output layer. Così da lasciare fuori le variabili negli hidden layer 1 e 2. Successivamente passiamo questa lista ristretta di variabili alla funzione *minimize* dell'optimizer. Ecco fatto! I layer 1 e 2 sono congelati: non saranno smossi durante la fase di addestramento, da qui il nome (*frozen layers*).

Esempio Completo:

In [0]:
#** 1. Costruzione del Modello **

reset_graph()

n_inputs= 28*28 #MNIST input size
n_hidden1 = 300 #riusati
n_hidden2 = 50 #riusati
n_hidden3 = 50 #riusati
n_hidden4 = 20 #nuovo
n_outputs = 10 #nuovo


#** 2. Organizzazione degli input e output **

X = tf.placeholder(tf.float32, shape=(None,n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

#** 3. Definizione degli scope e creazione dei layers **

with tf.name_scope("dnn"):
  hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.elu, name="hidden1") #riusato/freeze
  hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.elu, name="hidden2") #riusato/freeze
  hidden3 = tf.layers.dense(hidden2, n_hidden3, activation=tf.nn.elu, name="hidden3") #riusato
  hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.elu, name="hidden4") #nuovo
  logits = tf.layers.dense(hidden4, n_outputs, name="outputs") #nuovo
  
#** 4. Definizione della loss function **

with tf.name_scope("loss"):
  xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
  loss = tf.reduce_mean(xentropy, name="loss")
  
#** 5. Definizione della metrica di valutazione del modello **

with tf.name_scope("eval"):
  correct = tf.nn.in_top_k(logits,y,1)
  accuracy = tf.reduce_mean(tf.cast(correct,tf.float32), name="accuracy")
  
#** 6. Definizione calcolo gradiente e del training operation **

with tf.name_scope("train"):
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  #Codice nuovo usato per il freeze dei layers
  train_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 
                                 scope="hidden[34]|outputs")
  training_op = optimizer.minimize(loss, var_list=train_vars)
  #Fine parte nuova
  
#** 7. Inizializzazione delle variabile e creazione del Saver **

init = tf.global_variables_initializer()
new_saver = tf.train.Saver()

#** 8. Ripristino dei layers dal modello originale **

reuse_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,
                               scope="hidden[123]") # regular expression
restore_saver = tf.train.Saver(reuse_vars) # per ripristino dei layers 1-3
  
#** 9. Esecuzione del training della rete **

with tf.Session() as sess:
  init.run()
  restore_saver.restore(sess, "./my_model_final.ckpt")
  
  for epoch in range(n_epochs):
    for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
      sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
    
    accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
    print("Epoch: ",epoch, "Validation accuracy: ",accuracy_val)
  
  save_path = new_saver.save(sess, "./my_new_model_final.ckpt")

Un'altra opzione per congelare i layer inferiori è quella dell'aggiunta di un layer *stop_gradient()* nel grafo. Tutti i layer al di sotto saranno congelati: 
*Nota: Il codice è identico a quello dell'esempio precedente, va rimossa solo la parte nella definizione del training operation con l'aggiunta del nuovo layer nella creazione dei layer*

In [0]:
# [1.2.]

#** 3. Definizione degli scope e creazione dei layers **

with tf.name_scope("dnn"):
  hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.elu, name="hidden1") #riusato/freeze
  hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.elu, name="hidden2") #riusato/freeze
  #Aggiunta layer di stop, tutto quello al di sotto del layer 2 è congelato!
  hidden2_stop = tf.stop_gradient(hidden2)
  hidden3 = tf.layers.dense(hidden2_stop, n_hidden3, activation=tf.nn.elu, name="hidden3") #riusato, no freeze
  hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.elu, name="hidden4") #nuovo
  logits = tf.layers.dense(hidden4, n_outputs, name="outputs") #nuovo

# [4.5.]
  
#** 6. Definizione calcolo gradiente e del training operation **

with tf.name_scope("train"):
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  training_op = optimizer.minimize(loss)
  
# [7.8.9.]

## Modifica, Diminuzione o Sostituzione dei layer superiori

L'output layer del modello originale è tipicamente sostituito, non essendo molto utile per i nuovi task.
Similmente anche gli hidden layer superiori del modello originale sono meno utili rispetto a quelli inferiori, poichè le componenti superiori (High-Level features) risultano molto più utili per il nuovo task e possono differire significativamente da quelli che erano più utili per il task originale. Bisogna trovare il giusto numero di layer da riusare.

Per prima cosa si prova a congelare tutti i layer copiati del modello originale, si addestra il modello, guardandone la performance. 
Successivamente si scongelano uno o due layer appartenenti agli strati superiori, in modo da permettere al backpropagation di modificarli e si controlla se la performance è migliorata. Più dati si hanno a disposizione più layers è possibile scongelare.

Se la performance risulta ancora scadente e si hanno pochi dati a disposizione, si prova a diminuire gli hidden layer superiori (dropping) e congelare i restanti. Si itera fin quando non si trova il giusto numero di layer da riusare.

Se si hanno a disposizione molti dati per il training, si sostituiscono gli hidden layer superiori invece di diminuirli e nell'eventualità è possibile aggiungere altri hidden layer.


## Pre-Addestramento non supervisionato

Supponiamo di voler affrontare un task complesso di cui non si hanno a disposizione molti dati etichettati (labeled training data), ma sfortunatamente non si riesce a trovare un modello già addestrato per una task simile. Cosa si fa?
Primo, ovviamente si cerca di ottenere più dati possibili, ma se risulta molto dispendioso in termini di risorse e tempo, si può pensare di eseguire un *unsupervised pretraining*. Come illustrato dalla figura: 

![alt text](https://i.gyazo.com/990bde4b813d2f3581ca578cb54171e9.jpg)

Se i dati non classificati (unlabeled training data) sono in abbondanza, è possibile addestrare i layer uno ad uno, iniziando da quello più in basso e salire, usando un algoritmo di unsupervised feature detector (algoritmo di riconoscimento di componenti non supervisionato) come *Restricted Boltzmann Machines (RBMs)* o autoencoder.

Ogni layer è addestrato sull'output del layer precedentemente addestrato (tutti i layer ad eccezione del layer che si sta addestrando, sono congelati).
Una volta che tutti i layer sono stati addestrati in questo modo, si possono fare gli ultimi ritocchi alla rete usando il supervised learning (ad esempio, con il backpropagation). Può sembrare un processo lungo e dispendioso ma funziona molto bene. 
Tuttavia il pretraining non supervisionato (oggi si utilizzano gli autoencoders invece della RBMs) rimane comunque una buona opzione se si ha un task complesso da risolvere e non si ha un modello da riusare con i pochi dati a disposizione etichettati, ma si ha invece un buon numero di dati non etichettati (unlabeled training data).

# Faster Optimizers

Addestrare una rete neurale con milioni di parametri e migliaia di hidden layer può essere un processso maledettamento lento. Nella prima parte abbiamo visto 4 modi per velocizzare questo processo e raggiungere una buona soluzione:
1. Applicare una buona strategia di inizializzazione dei pesi.
2. Usare una buona funzione di attivazione che eviti i problemi di saturazione.
3. Usare la tecnica di Batch Normalization.
4. Riusare parti di reti già addestrate.

Un altro boost all'addestramento viene dall'uso di un optimizer veloce rispetto al classico *Gradient Descent optimizer*. Che sono:
Momentum optimization, Nesterov Accelerated Gradient, AdaGrad, RMSProp e infine Adam optimization (scoperto di recente ha avuto molto successo, infatti è quello più utilizzato attualmente).

## Momentum Optimization

In [0]:
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate,
                                       momentum=0.9)

## Nesterov Accelerated Gradient

In [0]:
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate,
                                       momentum=0.9, use_nesterov=True)

## AdaGrad

In [0]:
optimizer = tf.train.AdagradOptimizer(learning_rate=learning_rate)

## RMSProp

In [0]:
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate,
                                      momentum=0.9, decay=0.9, epsilon=1e-10)

## Adam Optimization

In [0]:
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

# Scheduling del Learning Rate

Trovare un buon learning rate può essere difficile. Se il valore impostato è molto alto, la loss function diverge, se è troppo basso, la funzione loss convergerà al minimo ottimale, ma i tempi saranno molto elevati. Se il valore è leggermente alto, si hanno dei progressi rapidamente, ma alla fine ci sarà una sorta di danza nell'intorno del minimo, dove la loss function non riuscirà a stabilirsi (a meno che non si utilizzi un algorimo di ottimizzazione come AdaGrad,RMSProp, o Adam, ma anche con questi, passerà del tempo prima di stabilizzarsi).

Se si ha a disposizione un budget computazionale limitato, l'addestramento può essere interrotto prima della convergenza, producendo una soluzione non ottimale però. Ecco una figura illustrata:

![alt text](https://i.gyazo.com/826628bad3a6ef6da967bfe5426d07d1.jpg)

Si può trovare un learning rate abbastanza buono, addestrando la rete più volte durante le epoch, variando il valore del learning rate e comparando le curve risultanti (valore della loss function in funzione del numero di epoch). 

Tuttavia è possibile fare di meglio che utilizzare un learning rate costante: se si utilizzasse un learning rate alto e si riducesse nel momento in cui smette di fare progressi in modo rapido, si può avere una buona soluzione, più veloce rispetto a quella di utilizzare un learning rate costante. Questa strategia è chiama *learning schedules*. Vediamo ora quali sono gli scheduling più comuni e quelli utilizzati effettivamente:

* *Valore di learning rate predeterminato costante*: Il learning rate è impostato con n0 = 0.1 all'inizio, dopo 50 epoch a  n1 = 0.001. Questa soluzione funziona molto bene, richiede solo la ricerca del learning rate giusto.

* *Performance Scheduling*: Si misura l'errore della loss function sul validation set ogni N step e si riduce il learning rate di un fattore pari a gamma quando l'errore smette di diminuire.

* *Exponential Scheduling*: Il learning rate è impostato come funzione del numero di iterazioni t: n(t) = n0\*10^(-t/r). Funziona  bene, ma richiede di impostare n0 e r. Il learning rate diminuirà (drop) di un fattore pari a 10 ogni r step.

* *Power Scheduling*: Learning rate è impostato come n(t) = n0\*(1 + t/r)^-c. L'hyperparametro c è tipicamente 1. E' simile all'exponential scheduling, ma il learnint rate diminuisce molto più lentamente.

La performance dei vari scheduling è stata comparata da *Andrew Senior* in un articolo durante l'addestramento di una rete neurale per lo speech recognition, usando il Momentum Optimization.
L'autore arriva alla conclusione che con entrambi gli scheduling di *performance* e *exponential* si hanno dei risultati ottimali, anche se è preferibile utilizzare *exponential* perchè molto facile da implementare, modificare (tune) e converge leggermente più veloce alla soluzione ottimale.

Ecco un esempio su come implementare il learning schedule in TensorFlow:

In [0]:
# [1.2.3.4.5]
  
#** 6. Definizione, calcolo gradiente e del training operation con learning rate**

with tf.name_scope("train"):
  
  initial_learning_rate = 0.1 #n0
  
  decay_steps = 10000 #r
  
  decay_rate = 1/10
  
  global_step = tf.Variable(0, trainable=False, name="global_step")
  
  learning_rate = tf.train.exponential_decay(initial_learning_rate, global_step
                                            ,decay_steps, decay_rate)
  
  optimizer = tf.train.MomentumOptimizer(learning_rate, momentum=0.9)
  
  training_op = optimizer.minimize(loss, global_step=global_step)
  
# [7.8.9.]

Dopo aver impostato i valori degli hyperparametri, si crea una variabile non trainable *global_step* inizializzata a 0, per tenere traccia del numero di iterazione corrente durante il training. Definiamo successivamente la variabile *learning_rate* come exponential decay (con n0=0.1 e r=10000), usando la funzione *exponential_decay()* di tensorflow.
Creiamo un optimizer usando il learning rate. Infine, il training operation chiamando il metodo *minimize()*, e passando oltre alla loss function, la variabile *global_step* che si occuperà di incrementare.

AdaGrad,RMSProp, e Adam Optimization automicamente riducono il learning rate durante il training, senza il bisogno di aggiungere una scheduling extra.
Per gli altri algoritmi di ottimizzazione, usando exponential decay (decadimento dell'esponente) può considerevolmente accelerare il processo di convergenza.